### Create DB connection and cursor

In [14]:
import sys
sys.path.append('/Users/nezmi/Projects/yeznable_projects')

from LAB.Data_collection import functions_for_db
from personal import id_pw_classes

# DB에 연결해놓기
# 로그인 정보들 숨길 필요가 있음
mysql_user = id_pw_classes.mysql()

host = '127.0.0.1'
user = mysql_user.id_yeznable
password = mysql_user.pw_yeznable
db = 'BigCon'

In [19]:
connection = functions_for_db.get_connection(host,user,password,db)
cursor = connection.cursor()
print('DB connected...')

DB connected...


### Data upload

In [ ]:
for i, row in enumerate(df.fillna(0).iterrows()):
#   Schema: ('방송일시','노출','마더코드','상품코드','상품명','상품군','판매단가','취급액')
    product_name = row[1][4].replace("'","")
    query_insert_data = f"""INSERT INTO SALES VALUES ({i+1},'{row[1][0]}',{row[1][1]},{row[1][2]},{row[1][3]},'{product_name}','{row[1][5]}',{row[1][6]},{row[1][7]})"""

    try:
        cursor.execute(query_insert_data)
        connection.commit()
    except:
        print(query_insert_data)

### Missing Values in column '노출'

In [16]:
# 방송일시와 노출시간을 가져와 딕셔너리로 만든다
query_select_columns = f"""SELECT 방송일시, 노출 FROM SALES ORDER BY 방송일시"""
cursor.execute(query_select_columns)
tuple_columns = cursor.fetchall()      # ((방송일시, 노출), ...)

dic_items = {}  # gonna be {방송일시: [노출, 품목수], ... }
for item in tuple_columns:
    try:
        dic_items[item[0]] = [max(dic_items[item[0]][0], item[1]), dic_items[item[0]][1] + 1]
    except:
        dic_items[item[0]] = [item[1], 1]

# for key in dic_items.keys():
#     print(key)
        
# dic_items

{datetime.datetime(2019, 1, 1, 6, 0): [20, 2],
 datetime.datetime(2019, 1, 1, 6, 20): [20, 2],
 datetime.datetime(2019, 1, 1, 6, 40): [20, 2],
 datetime.datetime(2019, 1, 1, 7, 0): [20, 1],
 datetime.datetime(2019, 1, 1, 7, 20): [20, 1],
 datetime.datetime(2019, 1, 1, 7, 40): [20, 1],
 datetime.datetime(2019, 1, 1, 8, 0): [20, 1],
 datetime.datetime(2019, 1, 1, 8, 20): [20, 1],
 datetime.datetime(2019, 1, 1, 8, 40): [20, 1],
 datetime.datetime(2019, 1, 1, 9, 0): [20, 1],
 datetime.datetime(2019, 1, 1, 9, 20): [20, 1],
 datetime.datetime(2019, 1, 1, 9, 40): [20, 1],
 datetime.datetime(2019, 1, 1, 10, 0): [20, 1],
 datetime.datetime(2019, 1, 1, 10, 20): [20, 1],
 datetime.datetime(2019, 1, 1, 10, 40): [20, 1],
 datetime.datetime(2019, 1, 1, 11, 0): [20, 1],
 datetime.datetime(2019, 1, 1, 11, 20): [20, 1],
 datetime.datetime(2019, 1, 1, 12, 0): [20, 1],
 datetime.datetime(2019, 1, 1, 12, 20): [20, 1],
 datetime.datetime(2019, 1, 1, 12, 40): [20, 1],
 datetime.datetime(2019, 1, 1, 13, 0): 

In [ ]:
# 방법1. 같은 시간에 방송한 다수의 제품에 대해 같은 노출시간을 지정
for key in dic_items.keys():
    query_update_missing_values_way1 = f"""UPDATE SALES SET 노출_1 = {dic_items[key][0]} WHERE 방송일시 = '{key}'"""
    cursor.execute(query_update_missing_values_way1)
    connection.commit()
#     print(query_update_missing_values_way1)
#     break

In [18]:
# 방법2. 같은 시간에 방송한 다수의 제품에 노출시간을 똑같이 나눠서 지정
for key in dic_items.keys():
    query_update_missing_values_way2 = f"""UPDATE SALES SET 노출_2 = {dic_items[key][0]/dic_items[key][1]} WHERE 방송일시 = '{key}'"""
    cursor.execute(query_update_missing_values_way2)
    connection.commit()
#     print(query_update_missing_values_way2)
#     break

UPDATE SALES SET 노출_1 = 10.0 WHERE 방송일시 = '2019-01-01 06:00:00'


In [ ]:
connection.close()